## Import Library

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

## Import Dataset

In [2]:
df = pd.read_excel('UAS_RFM.xlsx')
df.head()

,CardID,TrxDate,Amount
0,1000152,2021-05-19,932841
1,1000158,2021-03-25,881117
2,1000156,2021-04-25,368989
3,1000115,2021-06-14,621767
4,1000164,2021-01-02,451551


## Preparing the data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 KB


In [4]:
df.isnull().sum()

CardID     0
TrxDate    0
Amount     0
dtype: int64

In [5]:
df.shape

(199, 3)

## RFM Analysis

In [6]:
max_date = max(df['TrxDate'])
max_date

Timestamp('2021-06-30 00:00:00')

In [7]:
df['Diff'] = max_date - df['TrxDate']
df.head()

,CardID,TrxDate,Amount,Diff
0,1000152,2021-05-19,932841,42 days
1,1000158,2021-03-25,881117,97 days
2,1000156,2021-04-25,368989,66 days
3,1000115,2021-06-14,621767,16 days
4,1000164,2021-01-02,451551,179 days


### Get Recency Value

In [8]:
recency = df.groupby('CardID')['Diff'].min().reset_index()
recency.head()

,CardID,Diff
0,1000101,87 days
1,1000102,18 days
2,1000103,30 days
3,1000104,42 days
4,1000105,0 days


In [9]:
recency['Diff'] = recency['Diff'].dt.days
recency.columns = ['CardID','Recency']
recency.head()

,CardID,Recency
0,1000101,87
1,1000102,18
2,1000103,30
3,1000104,42
4,1000105,0


### Get Frequency Value

In [10]:
frequency = df.groupby('CardID')['TrxDate'].count().reset_index()
frequency.columns = ['CardID', 'Frequency']
frequency.head()

,CardID,Frequency
0,1000101,1
1,1000102,3
2,1000103,1
3,1000104,2
4,1000105,5


### Get Monetary Value

In [11]:
monetary = df.groupby('CardID')['Amount'].sum().reset_index()
monetary.columns = ['CardID', 'Monetary']
monetary.head()

,CardID,Monetary
0,1000101,203955
1,1000102,2120942
2,1000103,696250
3,1000104,612707
4,1000105,2590460


### Merge Recency. Frequency and Monetary

In [12]:
rfm = recency.merge(frequency,how='inner',on=['CardID'])
rfm = rfm.merge(monetary,how='inner',on=['CardID'])
rfm.head()

,CardID,Recency,Frequency,Monetary
0,1000101,87,1,203955
1,1000102,18,3,2120942
2,1000103,30,1,696250
3,1000104,42,2,612707
4,1000105,0,5,2590460


### RFM Quartiles

In [13]:
rfm_only = rfm.drop(['CardID'], axis=1)
rfm_only

,Recency,Frequency,Monetary
0,87,1,203955
1,18,3,2120942
2,30,1,696250
3,42,2,612707
4,0,5,2590460
...,...,...,...
66,24,1,657298
67,12,2,828660
68,28,1,748103
69,20,3,1393257


In [14]:
quantiles = rfm_only.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary
0.25,26.5,2.0,722176.5
0.50,44.0,3.0,1209115.0
0.75,88.0,3.0,2005925.0


In [15]:
quantiles.to_dict()

{'Recency': {0.25: 26.5, 0.5: 44.0, 0.75: 88.0},
 'Frequency': {0.25: 2.0, 0.5: 3.0, 0.75: 3.0},
 'Monetary': {0.25: 722176.5, 0.5: 1209115.0, 0.75: 2005925.0}}

### Create RFM Segments

In [16]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [17]:
rfm_segmentation = rfm
rfm_segmentation['R'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

rfm_segmentation.head()

,CardID,Recency,Frequency,Monetary,R,F,M
0,1000101,87,1,203955,2,1,1
1,1000102,18,3,2120942,4,2,4
2,1000103,30,1,696250,3,1,1
3,1000104,42,2,612707,3,1,1
4,1000105,0,5,2590460,4,4,4


### RFM Score

In [18]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R.map(str) \
                            + rfm_segmentation.F.map(str) \
                            + rfm_segmentation.M.map(str)
rfm_segmentation.head()

,CardID,Recency,Frequency,Monetary,R,F,M,RFMScore
0,1000101,87,1,203955,2,1,1,211
1,1000102,18,3,2120942,4,2,4,424
2,1000103,30,1,696250,3,1,1,311
3,1000104,42,2,612707,3,1,1,311
4,1000105,0,5,2590460,4,4,4,444


In [19]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False)

,CardID,Recency,Frequency,Monetary,R,F,M,RFMScore
50,1000152,2,7,3879215,4,4,4,444
41,1000143,5,7,3800039,4,4,4,444
9,1000110,26,5,3087323,4,4,4,444
14,1000115,10,4,2997985,4,4,4,444
4,1000105,0,5,2590460,4,4,4,444
55,1000157,8,6,2578668,4,4,4,444


### Summary From RFM Score

In [20]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='344']))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Best Customers:  6
Loyal Customers:  5
Big Spenders:  18
Almost Lost:  2
Lost Customers:  0
Lost Cheap Customers:  5
